## Variable Quality Survey and Cleaning

*18dec2017* 

#### Preliminary comments

In this data set, there is a good deal of marginal data with very spotty coverage.  It would simplify matters greatly to drop everything that isn't relevant to the main thrust of the project (model cost of residential solar installation as a function of time).  

The disadvantage of agressively removing data is that some interesting stories in the data might be lost.

Here are some examples questions I'd like to look at, having walked through the variables in the inital pass:

* What do the 100 largest installations look like.  Is their data reasonable?  Are they responsible for some of the out-sized ranges we see in distributions?  What is their cost/watt?  Over time?

* Compare costs for residential, commercial, government, etc.

* Installer
    * Installer trajectories over time (counts)
    * installer cost vs median cost over time

* Third part cost vs direct cost

There are some questions that are interesting but have very little data to work with:
* Tracking systems vs fixed systems
* BIPV systems vs roof mount
    
    
* Appraised cost vs unappraised costs

Modeling the cost of residential solar system will require filtering until we have a clean set of
date, cost and location data for a reasonable number of installations.

In this notebook, we will retain a quite liberal policy data so that we can look at some of the questions above with EDA. 

Prior to modeling, we will be more stringent about restricting the data to PV installations that are most pertinent to the overall modeling goal.

### Data Quality

**NOTE ALL MODIFICATIONS** to dfFull master data frame.  

#### First  cleaning steps

*   I know there are some bad indices from working with this data.  First get rid of these.
*  'type' is useless.  drop.  done.
*   downcase the 'install_type' column.  done.
        * can do more to clean up; there are a bunch of mis-categorizations
*  'zip_code' needs to be (valid) 5 character string in all cases 
        * small number of installs in MA have 00000 as zip
*  'state' needs cleaning - some state have a trailing blank.  fixed .
*  'new_constr', 'tracking' and '3rd_party' could perhaps be boolean.  On the other side of this issue NaN may mean exactly no available information.  We can use non-nullness as indicator of the presence of these characteristics. 
*   change '3rd_party'to third_party



##### Contents
1. imports
1. load csv
1. look at columns
1. build dict of cols
1. pull out mixed type cols
1. df info - 39 columns
1. df peek
1. parse date_installed
1. check for nulls
1. look at rows with null dates
1. get indices of null dates
1. drop rows with null dates
1. check again
1. index by date
1. peek at data
1. look at type column
1. drop type col
1. install_type value counts
1. downcase all
1. value_counts again
1. state value_counts
1. how many 'OH '
1. change them to 'OH'
1. check
1. how many 'MD '
1. change them to 'MD'
1. check
1. how many 'PA '
1. change to 'PA'
1. check
1. check all states have 2 chars
1. how many states
1. zipcode - how many of what types
1. look at zipcode data
1. change zip to 5 character strings
1. make sure they're all ok
1. look at high and low zips
1. how many '00000'? one row
1. look at the row with zipcode '00000' 
1. how many zipcodes < 501? 16
1. look at them; all Massachusetts
1. save data as csv

In [1]:
# set up
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

### Load data; set up indexing

In [2]:
# don't index yet
dfFull = pd.read_csv('../local/data/live_20171219.csv')

C:\Users\saus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,10,20,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# list columns
dfFull.columns

Index(['date_installed', 'state', 'incentive_prog_names', 'type', 'size_kw',
       'appraised', 'zipcode', 'install_type', 'installer', 'cost_per_watt',
       'cost', 'lbnl_tts_version_year', 'lbnl_tts', 'city', 'utility_clean',
       'tech_1', 'model1_clean', 'county', 'annual_PV_prod',
       'annual_insolation', 'rebate', 'sales_tax_cost', 'tilt1',
       'tracking_type', 'azimuth1', 'manuf2_clean', 'manuf3_clean',
       'manuf1_clean', 'inv_man_clean', 'reported_annual_energy_prod', 'year',
       'pbi_length', 'utility', 'bipv_3', 'bipv_2', 'bipv_1', 'new_constr',
       'tracking', '3rdparty'],
      dtype='object')

In [4]:
funky = dict(enumerate(dfFull.columns))

In [5]:
#####  Flags mixed type columns
[funky[i] for i in (4,10,20,30,32)]

['size_kw', 'cost', 'rebate', 'year', 'utility']

In [6]:
# tell me more
dfFull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020524 entries, 0 to 1020523
Data columns (total 39 columns):
date_installed                 1020516 non-null object
state                          1020524 non-null object
incentive_prog_names           797960 non-null object
type                           1020518 non-null object
size_kw                        1020519 non-null object
appraised                      224039 non-null object
zipcode                        1020519 non-null float64
install_type                   977940 non-null object
installer                      702466 non-null object
cost_per_watt                  762941 non-null float64
cost                           763105 non-null object
lbnl_tts_version_year          797958 non-null float64
lbnl_tts                       797959 non-null object
city                           798955 non-null object
utility_clean                  792718 non-null object
tech_1                         580918 non-null object
model1_clean   

In [7]:
dfFull.head()

,date_installed,state,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,reported_annual_energy_prod,year,pbi_length,utility,bipv_3,bipv_2,bipv_1,new_constr,tracking,3rdparty
0,1909-07-07,IN,NaN,uir,4.9,NaN,47401.0,Residential,Stumpner,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1909-07-07,IN,NaN,uir,7.35,NaN,47401.0,Residential,Stumpner,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1909-07-07,IN,NaN,uir,3.9,NaN,47408.0,Residential,Stumpner,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1909-07-07,IN,NaN,uir,9000,NaN,46222.0,Commercial,JMS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1969-12-31,FL,NaN,uir,5.52,NaN,32653.0,Residential,NaN,5.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### get ready to index by date_installed 

In [8]:
# parse the date
dfFull.date_installed = pd.to_datetime(dfFull.date_installed, infer_datetime_format=True)

In [9]:
# check for bad dates
dfFull.date_installed.isnull().sum()

8

In [10]:
# look at the rows
dfFull[dfFull.date_installed.isnull()]
# First three look misaligned; last 6 only have a state.
# These rows are bad, drop them.

,date_installed,state,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,reported_annual_energy_prod,year,pbi_length,utility,bipv_3,bipv_2,bipv_1,new_constr,tracking,3rdparty
863155,NaT,Los Angeles Department of Water & Power,Mono,OPT325-72-4-100,Los Angeles,707151.5436,5.560624,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898055,NaT,Los Angeles Department of Water & Power,NaN,NaN,Los Angeles,255673.1799,5.572553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
928210,NaT,Los Angeles Department of Water & Power,Poly,CS6P-250P,Los Angeles,172936.1267,5.560624,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020519,NaT,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020520,NaT,OH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020521,NaT,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020522,NaT,OH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020523,NaT,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# check correct syntax for drop
dfFull[dfFull.date_installed.isnull()].index

Int64Index([863155, 898055, 928210, 1020519, 1020520, 1020521, 1020522,
            1020523],
           dtype='int64')

In [12]:
# okay, do it.
dfFull.drop(dfFull[dfFull.date_installed.isnull()].index, inplace=True)

In [13]:
# check again to make sure
dfFull.date_installed.isnull().sum()
# okay all gone.

0

#### Okay, ready to index by date_installed

In [14]:
# now index by date_installed
dfFull.set_index('date_installed', drop=True, inplace=True)

In [15]:
dfFull.head()

,state,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,cost,...,reported_annual_energy_prod,year,pbi_length,utility,bipv_3,bipv_2,bipv_1,new_constr,tracking,3rdparty
date_installed,,,,,,,,,,,,,,,,,,,,,
1909-07-07,IN,NaN,uir,4.9,NaN,47401.0,Residential,Stumpner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909-07-07,IN,NaN,uir,7.35,NaN,47401.0,Residential,Stumpner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909-07-07,IN,NaN,uir,3.9,NaN,47408.0,Residential,Stumpner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909-07-07,IN,NaN,uir,9000,NaN,46222.0,Commercial,JMS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-12-31,FL,NaN,uir,5.52,NaN,32653.0,Residential,NaN,5.9,32568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Examine columns - part i: useful fields

#### Size and cost
```
size_kw                        1020516 non-null float64
cost_per_watt                  762941 non-null float64
cost                           763102 non-null float64
```

#### Geography
```
state                          1020521 non-null object
zipcode                        1020516 non-null float64
city                           798954 non-null object
county                         998652 non-null object
```

#### Other
```
type                           1020516 non-null object
install_type                   977940 non-null object
installer                      702466 non-null object
new_constr                     27106 non-null float64
tracking                       1930 non-null float64
3rdparty                       306993 non-null float64
```

#### Type column
This is not described in LBNL/NREL document.  Can't find these as acronyms in any glossary.  May be internal to LBNL/NREL.  Choosing to drop this column now.

In [16]:
# look at type column
dfFull.type.value_counts()

uir    1020005
pir        511
Name: type, dtype: int64

In [17]:
### **MOD**  dropping 
dfFull.drop('type', axis='columns', inplace=True)

#### install_type column - mixed case

Cleaned up the case problems but there are still some hokey installation types.  Some could be sensibly consolidated into larger type, but hard to tell about some of them.

In [18]:
dfFull.install_type.value_counts()

residential                    728661
Residential                    179321
commercial                      24470
Unknown                         15771
Commercial                      11908
unknown                          4034
government                       3721
nonprofit                        3003
Government                       2027
educational                      1703
Nonprofit                        1217
Educational                       710
Customer                          371
education                         290
Agricultural                      284
Utility                           138
agricultural                      124
utility                            92
Institutional                      29
Commercial - Other                 16
Public                             11
Gov't/NP                           11
Residential/SF                      5
public                              5
agriculture                         5
Nonresidential                      3
Commercial -

In [19]:
### **MOD** downcase all
dfFull.install_type = dfFull.install_type.str.lower()

In [20]:
dfFull.install_type.value_counts()

residential                    907982
commercial                      36378
unknown                         19805
government                       5748
nonprofit                        4220
educational                      2413
agricultural                      408
customer                          371
education                         290
utility                           230
institutional                      29
public                             16
commercial - other                 16
gov't/np                           11
residential/sf                      5
agriculture                         5
nonresidential                      3
not stated                          2
commercial - small business         2
small business                      2
commercial - agriculture            1
commercial - builders               1
commerical                          1
municipal                           1
Name: install_type, dtype: int64

#### State column - make all states 2 letter strings

In [21]:
dfFull.state.value_counts()

CA     626820
AZ      89361
MA      71421
NJ      51046
NY      51005
CT      23643
NV      16048
TX      14347
PA      11430
MD      11419
NM       8356
MO       5504
DE       5119
OR       4280
NH       3501
WI       3321
DC       3296
FL       2984
CO       2606
HI       2324
OH       2259
VT       1642
MN       1570
IL       1484
IN        850
UT        817
ME        702
TN        601
RI        583
LA        345
MI        263
AR        239
WA        209
IA        206
WY        157
SC        132
WV         92
NC         75
MS         71
MT         61
VA         55
GA         53
ID         44
KY         33
AL         32
KS         28
AK         22
NE         17
SD         16
OK         10
OH          6
PR          4
ND          3
PA          2
MD          2
Name: state, dtype: int64

In [22]:
# extra spaces
(dfFull.state == 'OH ').sum()

6

In [23]:
# **MOD** get rid of extra space
dfFull.loc[dfFull.state == 'OH ', 'state'] = 'OH'

In [24]:
# clean now
(dfFull.state == 'OH ').sum()

0

In [25]:
# extra spaces
(dfFull.state == 'MD ').sum()

2

In [26]:
# **MOD** get rid of extra space
dfFull.loc[dfFull.state == 'MD ', 'state'] = 'MD'

In [27]:
# check
(dfFull.state == 'MD ').sum()

0

In [28]:
# extra spaces?
(dfFull.state == 'PA ').sum()

2

In [29]:
# **MOD**
dfFull.loc[dfFull.state == 'PA ', 'state'] = 'PA'

In [30]:
# check
(dfFull.state == 'PA ').sum()

0

In [31]:
# okay, all states now have 2 letters, right??  Yes.
dfFull.state.apply(lambda s: True if len(s) != 2 else False).sum()

0

In [32]:
# extra 2 are DC and PR
dfFull.state.value_counts().size

52

#### Zipcode - convert to 5 character string

There are 15 broken zipcodes in MA.  Other data for these entries looks okay.  Leave in for now.

In [33]:
# there was a mixed type warning on load for this column.
zip_types = dfFull.zipcode.apply(type)
zip_types.value_counts()
# but it looks okay wonder what they did about leading 0s

<class 'float'>    1020516
Name: zipcode, dtype: int64

In [34]:
# okay, they lost the leading zeros (e.g. MA); doesn't seem like TEOTW.
dfFull.iloc[:20,:]

,state,incentive_prog_names,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,cost,lbnl_tts_version_year,...,reported_annual_energy_prod,year,pbi_length,utility,bipv_3,bipv_2,bipv_1,new_constr,tracking,3rdparty
date_installed,,,,,,,,,,,,,,,,,,,,,
1909-07-07,IN,NaN,4.9,NaN,47401.0,residential,Stumpner,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909-07-07,IN,NaN,7.35,NaN,47401.0,residential,Stumpner,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909-07-07,IN,NaN,3.9,NaN,47408.0,residential,Stumpner,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909-07-07,IN,NaN,9000,NaN,46222.0,commercial,JMS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-12-31,FL,NaN,5.52,NaN,32653.0,residential,NaN,5.90,32568,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-12-31,MA,NaN,2.88,NaN,2138.0,residential,NaN,6.94,20000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-12-31,MA,NaN,2.52,NaN,2138.0,residential,NaN,9.27,23351.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-12-31,MA,NaN,2.76,NaN,2474.0,residential,NaN,6.48,17878,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-12-31,MA,NaN,13.23,NaN,2139.0,unknown,NaN,7.24,95824,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# **MOD**
# '{:05d}'.format(123)
dfFull.zipcode = dfFull.zipcode.apply(lambda z:'{:05d}'.format(int(z)))

In [36]:
# 1020516 5 char strings zero padded
(dfFull.zipcode.apply(len) == 5).sum()

1020516

In [37]:
# look at low, high
(dfFull.zipcode.min(), dfFull.zipcode.max())
# oops, shouldn't be any all zeros; it's not a valid zipcode

('00000', '99737')

In [38]:
# Good.  There's only one.
(dfFull.zipcode == '00000').sum()

1

In [39]:
# Have a look 
dfFull[dfFull.zipcode == '00000']
# has state, size, zip(bad), install_type, cpw  and cost

,state,incentive_prog_names,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,cost,lbnl_tts_version_year,...,reported_annual_energy_prod,year,pbi_length,utility,bipv_3,bipv_2,bipv_1,new_constr,tracking,3rdparty
date_installed,,,,,,,,,,,,,,,,,,,,,
2012-08-01,MA,NaN,8.96,NaN,00000,residential,NaN,5.89,52800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### What is the lowest zip code in the United States?
see: https://www.google.com/search?q=what+is+range+of+valid+zip+codes&rlz=1C1CHBF_enUS747US747&oq=what+is+range+of+valid+zip+codes&aqs=chrome..69i57.9578j0j7&sourceid=chrome&ie=UTF-8

00501
The lowest ZIP code is in Holtsville, New York (00501, a unique ZIP Code for the U.S. Internal Revenue Service center there). Other low ZIP codes are 00601 for Adjuntas, Puerto Rico; 01001 for Agawam, Massachusetts, and 01002 for Amherst, Massachusetts.
ZIP Code History - ZIP Code Boundaries
www.zipboundary.com/zipcode_history.html


In [40]:
# let's look at everything below 501
(dfFull.zipcode.apply(int) < 501).sum()

16

In [41]:
# okay, I'm going to leave these in for now noting that I have 15 bogus zipcodes  in MA
# MA zipcodes do start with 0; assume it's a data entry error.  Other data while minimal
# looks okay
dfFull[dfFull.zipcode.apply(int) < 501]

,state,incentive_prog_names,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,cost,lbnl_tts_version_year,...,reported_annual_energy_prod,year,pbi_length,utility,bipv_3,bipv_2,bipv_1,new_constr,tracking,3rdparty
date_installed,,,,,,,,,,,,,,,,,,,,,
2011-07-20,MA,NaN,7.4,NaN,00242,residential,NaN,4.94,36587.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-29,MA,NaN,4.21,NaN,00244,residential,NaN,5.80,24418,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-01,MA,NaN,8.82,NaN,00276,residential,NaN,4.85,42777,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-01,MA,NaN,2.3,NaN,00206,residential,NaN,5.15,11848.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-02,MA,NaN,3.24,NaN,00264,residential,NaN,5.37,17404,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-11,MA,NaN,2.88,NaN,00253,residential,NaN,7.11,20477,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-11,MA,NaN,7.2,NaN,00253,residential,NaN,7.08,51000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-22,MA,NaN,5.04,NaN,00236,residential,NaN,6.04,30440,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-31,MA,NaN,4.7,NaN,00236,residential,NaN,4.82,22661.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## I want to save the dataset here, then continue with part 2.

In [42]:
# already done
# dfFull.to_csv('./data/live_20171219_1.csv')